# Binary Words → Decimal Lens → Tier‑1/Tier‑2 Gates (Quick Test)


This notebook builds classic binary words (Fibonacci/Rabbit, Thue–Morse, Period‑Doubling, Rudin–Shapiro) and the binary fractional digits of φ−1, then maps their 0/1 digits into a **base‑10 decimal** `x = 0.W`.
It applies all Tier‑1 gates (`comp`, `inv`, `odds`, `sign`) and all ordered Tier‑2 pairs, and logs the results.
Optionally applies a fixed coarse **scale** `S(x) = (729/1000) * x`.


In [1]:

# --- Parameters ---
SEQ_LEN = 256
INCLUDE_COMPLEMENTS = True
APPLY_SCALE_S = False

from decimal import Decimal, getcontext, DivisionByZero, InvalidOperation, ROUND_HALF_EVEN
import math, pandas as pd

getcontext().prec = 200
getcontext().rounding = ROUND_HALF_EVEN


In [2]:

def fib_word(n):
    s = "0"
    while len(s) < n:
        out = []
        for ch in s:
            out.append("01" if ch == "0" else "0")
        s = "".join(out)
    return [1 if ch == "1" else 0 for ch in s[:n]]

def thue_morse(n):
    def bitcount(k):
        c = 0
        while k:
            k &= k - 1
            c += 1
        return c
    return [bitcount(k) & 1 for k in range(n)]

def period_doubling(n):
    s = "0"
    while len(s) < n:
        out = []
        for ch in s:
            out.append("01" if ch == "0" else "00")
        s = "".join(out)
    return [1 if ch == "1" else 0 for ch in s[:n]]

def rudin_shapiro(n):
    def a(k):
        b = k
        prev = 0
        pairs = 0
        while b:
            cur = b & 1
            if cur and prev:
                pairs += 1
            prev = cur
            b >>= 1
        return pairs & 1
    return [a(k) for k in range(n)]

def phi_minus_one_binary_digits(n):
    phi = (1 + math.sqrt(5)) / 2.0
    y = 1.0 / phi
    digs = []
    for _ in range(n):
        y *= 2.0
        if y >= 1.0:
            digs.append(1)
            y -= 1.0
        else:
            digs.append(0)
    return digs

def complement_bits(bits):
    return [1 - b for b in bits]

def decimal_from_bits(bits):
    d = Decimal(0)
    ten = Decimal(10)
    pw = Decimal(1)
    for b in bits:
        pw = pw / ten
        if b:
            d += pw
    return d


In [3]:

def comp(x: Decimal): return Decimal(1) - x

def inv(x: Decimal):
    if x == 0:
        raise DivisionByZero
    return Decimal(1) / x

def odds(x: Decimal):
    one = Decimal(1)
    if x == one:
        raise DivisionByZero
    return x / (one - x)

def sign(x: Decimal): return -x

GATES = {"comp": comp, "inv": inv, "odds": odds, "sign": sign}

def apply_gate_safe(f, x: Decimal):
    try:
        return f(x)
    except (DivisionByZero, InvalidOperation, ZeroDivisionError, OverflowError):
        return Decimal('NaN')

def compose_safe(f, g):
    return lambda x: apply_gate_safe(g, apply_gate_safe(f, x))

def scale_S(x: Decimal):
    return (Decimal(729) / Decimal(1000)) * x


In [4]:

# Build seeds
sequences = {
    "FibonacciWord": fib_word(SEQ_LEN),
    "ThueMorse": thue_morse(SEQ_LEN),
    "PeriodDoubling": period_doubling(SEQ_LEN),
    "RudinShapiro": rudin_shapiro(SEQ_LEN),
    "PhiMinus1_BinaryDigits": phi_minus_one_binary_digits(SEQ_LEN),
}
if INCLUDE_COMPLEMENTS:
    sequences = {**sequences, **{name + "_COMP": complement_bits(bits) for name, bits in sequences.items()}}

tier1 = list(GATES.keys())
tier2_pairs = [(a,b) for a in GATES for b in GATES]

rows = []
for seq_name, bits in sequences.items():
    x = decimal_from_bits(bits)
    for g1 in tier1:
        v = apply_gate_safe(GATES[g1], x)
        rec = {"sequence": seq_name, "tier": "T1", "path": g1, "x_decimal": str(x), "value": str(v)}
        rows.append(rec)
    for g1,g2 in tier2_pairs:
        f = compose_safe(GATES[g1], GATES[g2])
        v = f(x)
        rec = {"sequence": seq_name, "tier": "T2", "path": f"{g1}→{g2}", "x_decimal": str(x), "value": str(v)}
        rows.append(rec)

df = pd.DataFrame(rows)
df.head(20)


,sequence,tier,path,x_decimal,value
0,FibonacciWord,T1,comp,0.01001010010010100101001001010010010100101001...,0.98998989989989899898998998989989989899898998...
1,FibonacciWord,T1,inv,0.01001010010010100101001001010010010100101001...,99.8991009080828362454828714825280940040314347...
2,FibonacciWord,T1,odds,0.01001010010010100101001001010010010100101001...,0.01011131537919038749889759160915543641309222...
3,FibonacciWord,T1,sign,0.01001010010010100101001001010010010100101001...,-0.0100101001001010010100100101001001010010100...
4,FibonacciWord,T2,comp→comp,0.01001010010010100101001001010010010100101001...,0.01001010010010100101001001010010010100101001...
5,FibonacciWord,T2,comp→inv,0.01001010010010100101001001010010010100101001...,1.01011131537919038749889759160915543641309222...
6,FibonacciWord,T2,comp→odds,0.01001010010010100101001001010010010100101001...,98.8991009080828362454828714825280940040314347...
7,FibonacciWord,T2,comp→sign,0.01001010010010100101001001010010010100101001...,-0.9899898998998989989899899898998998989989899...
8,FibonacciWord,T2,inv→comp,0.01001010010010100101001001010010010100101001...,-98.899100908082836245482871482528094004031434...
9,FibonacciWord,T2,inv→inv,0.01001010010010100101001001010010010100101001...,0.01001010010010100101001001010010010100101001...


In [5]:

# Optional: domain hit summary (NaN rate per path)
summary = (df.assign(is_nan = df["value"].apply(lambda s: (s == 'NaN')))
             .groupby(["tier","path"])["is_nan"].mean()
             .rename("nan_rate")
             .reset_index()
             .sort_values(["tier","nan_rate","path"]))
summary.head(20)


,tier,path,nan_rate
0,T1,comp,0.0
1,T1,inv,0.0
2,T1,odds,0.0
3,T1,sign,0.0
4,T2,comp→comp,0.0
5,T2,comp→inv,0.0
6,T2,comp→odds,0.0
7,T2,comp→sign,0.0
8,T2,inv→comp,0.0
9,T2,inv→inv,0.0


In [6]:

# Save outputs for easy download in Colab
import os
out_dir = "/content"
os.makedirs(out_dir, exist_ok=True)
csv_path = os.path.join(out_dir, "binary_words_transform_results.csv")
df.to_csv(csv_path, index=False)
print("Wrote:", csv_path)


Wrote: /content/binary_words_transform_results.csv
